In [2]:
# Import block
import pandas as pd
import numpy as np
import sklearn    # Comes with kNN method, may be useful

In [41]:
# Data loading block
# The MovieLens-20M dataset is split over 6 .csv files
# 1. genome-tags:    Associates each movie tag with a number
# 2. tags:           Each user and what tags they have assigned to which movies, with timestamp
# 3. genome-scores:  Each tag's relevance to every movie as a double 0-1
# 4. ratings:        20M ratings users gave to movies they watched (0.5-5, steps of 0.5)
# 5. movies:         Each movie's ID number and name, as well as genres 
# 6. links:          Each movie's entry numbers in IMDB and themovieDB (seemingly irrelevant?)

genome_tags = pd.read_csv("genome-tags.csv") # get tags
user_tags = pd.read_csv("tags.csv", usecols=[0, 1, 2]) # get user-given tags, but not timestamp
genome_scores = pd.read_csv("genome-scores.csv")
ratings = pd.read_csv("ratings.csv", usecols=[0, 1, 2]) # get user-provided ratings
movies = pd.read_csv("movies.csv")
# links = pd.read_csv("links.csv")

############# DATA SAVING STEP ######################
# Set movie IDs to int32 from int64
movies["movieId"] = movies["movieId"].astype('int32')

# Change ratings from floats (0.5 to 5 in 0.5 increments) to integers (1-10)
# Change user ID and movie ID to int32 from int64
ratings["userId"] = ratings["userId"].astype('int32')
ratings["movieId"] = ratings["movieId"].astype('int32')
ratings["rating"] = (ratings["rating"] * 2).astype('int8') # Keep this in mind (1-10)

# Change movieId to int32 and tagId to int16
# Change relevance from float64 to float32
genome_scores["movieId"] = genome_scores["movieId"].astype('int32')
genome_scores["tagId"] = genome_scores["tagId"].astype('int16')
genome_scores["relevance"] = genome_scores["relevance"].astype('float32')
                     
# Change userId and movieId to int32
user_tags["userId"] = user_tags["userId"].astype('int32')
user_tags["movieId"] = user_tags["movieId"].astype('int32')
                     
# Change genome tag IDs to int16
genome_tags["tagId"] = genome_tags["tagId"].astype('int16')

In [4]:
print("Items in genome-tags:\t\t" + str(len(genome_tags.index)))
print("Items in user-tags:\t\t" + str(len(user_tags.index)))
print("Items in genome-scores:\t\t" + str(len(genome_scores.index)))
print("Items in ratings:\t\t" + str(len(ratings.index)))
print("Items in movies:\t\t" + str(len(movies.index)))

Items in genome-tags:		1128
Items in user-tags:		465564
Items in genome-scores:		11709768
Items in ratings:		20000263
Items in movies:		27278


In [40]:
ratings["rating"]

0           3.5
1           3.5
2           3.5
3           3.5
4           3.5
5           3.5
6           4.0
7           4.0
8           4.0
9           4.0
10          4.0
11          4.0
12          4.0
13          3.5
14          3.5
15          4.0
16          3.5
17          3.5
18          3.0
19          3.5
20          3.5
21          3.5
22          4.0
23          4.0
24          3.5
25          3.5
26          4.0
27          4.0
28          3.5
29          3.5
           ... 
20000233    3.5
20000234    3.5
20000235    4.5
20000236    4.5
20000237    4.0
20000238    4.0
20000239    4.0
20000240    3.0
20000241    4.0
20000242    4.0
20000243    4.5
20000244    5.0
20000245    5.0
20000246    3.0
20000247    3.0
20000248    4.5
20000249    4.0
20000250    3.0
20000251    5.0
20000252    4.0
20000253    4.5
20000254    4.0
20000255    4.5
20000256    4.5
20000257    4.5
20000258    4.5
20000259    4.5
20000260    3.0
20000261    5.0
20000262    2.5
Name: rating, Length: 20

In [21]:
num_users = ratings["userId"].iloc[-1] # Get number of users (by looking at value of the final user)
# Create a user-movie matrix where rows=users and columns=movie ratings
#user_ratings_matrix = pd.DataFrame(index=range(0, num_users+1), columns=movies["movieId"])

#user_ratings_matrix.shape

# Notes: user tags are almost useless - many include their own and stray from the genome tags

# RATINGS appear important - other users' ratings on movies in the same GENRE would affect recs


## "12/11/2019"